In [1]:
%run ../Requirements.ipynb

24/06/20 15:46:57 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
24/06/20 15:46:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 15:46:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
response = requests.get(apple_music_itunes_url)
top_tracks = response.json()

track_list = []

def extract_artist_id(url):
    match = re.search(r'/artist/.+/(\d+)\?', url)
    return match.group(1) if match else None

for entry in top_tracks['feed']['entry']:
    artist_url = entry['im:artist']['attributes']['href']
    artist_id = extract_artist_id(artist_url)
    
    track_info = {
        'Track Name': entry['im:name']['label'],
        'Artist Name': entry['im:artist']['label'],
        'Album Name': entry['im:collection']['im:name']['label'],
        'Release Date': entry['im:releaseDate']['attributes']['label'],
        'Track URL': entry['link'][0]['attributes']['href'],
        'Artist ID': artist_id
    }
    track_list.append(track_info)

df = pd.DataFrame(track_list)
spark_df = spark.createDataFrame(df)

In [3]:
spark_df.write.mode('overwrite').format('parquet').save('../Staging/AppleMusic')

24/06/20 15:47:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
